In [1]:
from hcoefs_from_atk import hcoefs_unsorted

In [180]:
import os, json

In [3]:
os.chdir('../code/numbertheory/')

In [53]:
from utils import discfac, divisors, primesBetween,poly_eval_mod, quad_rec, gcd

In [5]:
def supp(p):
    a = 0
    ds = []
    while a*a < 4*p:
        ds.append(a*a-4*p)
        a+=1
    return ds

def d_clos(d):
    d0, c = discfac(d)
    return [d0*(c0**2) for c0 in divisors(c)]

def supp_clos(p):
    ds = []
    for d in supp(p):
        ds+=d_clos(d)
    ds = list(set(ds))
    ds.sort(key=abs)
    return ds

ssps = primesBetween(2,36)+[41]+[47+12*i for i in range(3)]
ssds = []
for p in ssps:
    ssds+=supp_clos(p)
ssds = list(set(ssds))
ssds.sort(key=abs)
disc_2_hcoefs = {-3:[0,1],-4:[-1728,1]}
ssds_unsorted = ssds[2:]
hcoefs_unsorted = [cs for cs in hcoefs_unsorted if cs[0] not in [0,-1728]]


In [6]:
def poly_eval_mod(coefs_lc_to_const:list[int],x:int,p:int):
    evx = coefs_lc_to_const[0]%p
    for c in coefs_lc_to_const[1:]:
        evx = (evx*x+c)%p
    return evx

def trace_frob(fg:tuple[int,int],p:int)->int:
    f,g = fg
    return - sum([quad_rec(x**3+f*x+g,p) for x in range(p)])

def j_to_disc(j:int,p:int)->int:
    if j%p == 0:
        if p % 3 == 1:
            return -3
        else:
            return -4*p
    elif (j-1728)%p == 0:
        if p % 4 == 1:
            return -4
        else:
            return -p
    else:
        f = -3*j*(j-1728)
        g = 2*j*(j-1728)**2
        a = trace_frob((f,g),p)
        return a**2 - 4*p
    

In [ ]:
data = {}
for i, cs in enumerate(hcoefs_unsorted):
    datai = {'coefs':cs,'primes':[],'js':[],'ds':[]}
    for p in primesBetween(5,400):
        cp = [c% p for c in cs]
        rtsp = [j0 for j0 in range(p) if poly_eval_mod(cp[::-1],j0,p)==0]
        if len(rtsp)==len(cs)-1:
            datai['primes'].append(p)
            datai['js'].append(rtsp)
            datai['ds'].append(j_to_disc(rtsp[0],p))
    data[i] = datai
h41 = hcoefs_unsorted.pop(52)
disc_2_hcoefs[-41*4]=h41
ssds_unsorted = [d for d in ssds if d not in disc_2_hcoefs]

In [20]:
data = {}
for i, cs in enumerate(hcoefs_unsorted):
    datai = {'coefs':cs,'primes':[],'js':[],'ds':[]}
    for p in primesBetween(5,512):
        cp = [c% p for c in cs]
        rtsp = [j0 for j0 in range(p) if poly_eval_mod(cp[::-1],j0,p)==0]
        if len(rtsp)==len(cs)-1:
            d = j_to_disc(rtsp[0],p)
            if d >=-284:
                datai['primes'].append(p)
                datai['js'].append(rtsp)
                datai['ds'].append(j_to_disc(rtsp[0],p))
    datai['ds_nr'] = list(set(datai['ds']))
    datai['ds_nr'].sort(key=abs)
    data[i] = datai

In [50]:
d_cands = {i:[] for i in range(len(hcoefs_unsorted))}
for i, cs in enumerate(hcoefs_unsorted):
    for p in ssps:
        cp = [c% p for c in cs]
        rtsp = [j0 for j0 in range(p) if poly_eval_mod(cp[::-1],j0,p)==0]
        if len(rtsp)==len(cs)-1:
            d = j_to_disc(rtsp[0],p)
            if d >=-284 and quad_rec(d,p)==1:
                d_cands[i].append(d)

for i in d_cands:
    di = d_cands[i]
    di = list(set(di))
    di.sort(key=abs)
    d_cands[i]=di

In [66]:
d_cands_gcd = {i:0 for i in range(len(hcoefs_unsorted))}
for i, cs in enumerate(hcoefs_unsorted):
    for p in ssps:
        cp = [c% p for c in cs]
        rtsp = [j0 for j0 in range(p) if poly_eval_mod(cp[::-1],j0,p)==0]
        if len(rtsp)==len(cs)-1:
            d = j_to_disc(rtsp[0],p)
            if quad_rec(d,p)==1:
                d_cands_gcd[i] =-gcd(d_cands_gcd[i],d)

d_to_cands_v2 ={d_cands_gcd[d]:[] for d in d_cands_gcd}

for d in d_cands_gcd:
    if d_cands_gcd[d]!= 0:
        d_to_cands_v2[d_cands_gcd[d]].append(d)

In [ ]:
tentative_assignments = {}
tentative_assignees = []
leftovers = []
for d in ssds_unsorted:
    if d in d_to_cands_v2 and len(d_to_cands_v2[d])==1:
        i_d = d_to_cands_v2[d][0]
        tentative_assignees.append(i_d)
        tentative_assignments[d]=data[i_d]['coefs']
    else:
        leftovers.append(d)

In [80]:
tentative_assignments

{-8: [-8000, 1],
 -11: [32768, 1],
 -12: [-54000, 1],
 -16: [-287496, 1],
 -19: [884736, 1],
 -20: [-681472000, -1264000, 1],
 -24: [14670139392, -4834944, 1],
 -27: [12288000, 1],
 -32: [12167000000, -52250000, 1],
 -35: [-134217728000, 117964800, 1],
 -36: [-1790957481984, -153542016, 1],
 -40: [9103145472000, -425692800, 1],
 -43: [884736000, 1],
 -44: [-653249011576832, 270413882112, -1122662608, 1],
 -48: [6549518250000, -2835810000, 1],
 -51: [6262062317568, 5541101568, 1],
 -52: [-567663552000000, -6896880000, 1],
 -56: [10064086044321563803648,
  2257767342088912896,
  2059647197077504,
  -16220384512,
  1],
 -59: [374643194001883136, -140811576541184, 30197678080, 1],
 -64: [-7367066619912, -82226316240, 1],
 -67: [147197952000, 1],
 -72: [232381513792000000, -377674768000, 1],
 -75: [5209253090426880, 654403829760, 1],
 -76: [-827237892283232256, 1128678666363648, -784074438864, 1],
 -80: [422286883970526784000000,
  -171263969177632000000,
  -13028555239824000,
  -1597177172

In [82]:
for d in tentative_assignments:
    disc_2_hcoefs[d] = tentative_assignments[d]

ssds_unsorted = [d for d in ssds if d not in disc_2_hcoefs]

In [86]:
len(ssds_unsorted)

17

In [88]:
ssds_unsorted_conds = {discfac(d)[0]:[] for d in ssds_unsorted}
for d in ssds_unsorted:
    d0,c = discfac(d)
    ssds_unsorted_conds[d0].append(c)

In [89]:
ssds_unsorted_conds

{-7: [1, 2],
 -15: [1, 2],
 -23: [1, 2],
 -31: [1, 2],
 -47: [1, 2],
 -55: [1, 2],
 -68: [1],
 -71: [1, 2],
 -116: [1],
 -59: [2]}

In [126]:
def count_2tor(j,p):
    return len([x for x in range(p) if ((x**3)-3*j*(j-1728)*x+2*j*(j-1728)**2)%p == 0])

In [129]:
c2unsortedpairs = {discfac(d):d_to_cands_v2[d] for d in leftovers if d in d_to_cands_v2}
cond2data = {}
for pair in c2unsortedpairs:
    d0 = pair[0]
    i0 = c2unsortedpairs[pair][0]
    i1 = c2unsortedpairs[pair][1]
    cs0 = data[i0]['coefs']
    cs1 = data[i1]['coefs']
    p0 = data[i0]['primes'][-1]
    j0 = data[i0]['js'][-1][0]
    j1 = data[i1]['js'][-1][0]
    cond2data[d0] = {'coefs':[cs0,cs1],'p':p0,'js':[j0,j1],'2tor':[count_2tor(j,p0) for j in [j0,j1]]}
    

In [154]:
remaining_indices_pass1 = [i for i in data if i not in tentative_assignees]
remaining_indices_pass2 = remaining_indices_pass1.copy()
for dc in c2unsortedpairs:
    pair = c2unsortedpairs[dc]
    remaining_indices_pass2 = [i for i in remaining_indices_pass2 if i not in pair]

In [155]:
remaining_indices_pass2

[22, 39, 55, 56, 67, 75, 76]

In [133]:
tent_assg_2 = {}
for d0 in cond2data:
    for i in [0,1]:
        t2i = cond2data[d0]['2tor'][i]
        if t2i == 1:
            tent_assg_2[4*d0] = cond2data[d0]['coefs'][i]
        else:
            tent_assg_2[d0] = cond2data[d0]['coefs'][i]

In [156]:
for d in tent_assg_2:
    disc_2_hcoefs[d] = tent_assg_2[d]

In [ ]:
ssds_unsorted_last = [d for d in ssds_unsorted if d not in tent_assg_2]


In [158]:
ps_for_last = [17,29,47,59,71]

In [159]:
{p:[d for d in ssds_unsorted_last if d % p == 0] for p in ps_for_last}

{17: [-68], 29: [-116], 47: [-47, -188], 59: [-236], 71: [-71, -284]}

In [171]:
{p:[i for i in remaining_indices_pass2 if poly_eval_mod(data[i]['coefs'][::-1],1728%p,p)==0] for p in ps_for_last}

{17: [], 29: [], 47: [55, 56, 67, 75], 59: [22, 67, 75], 71: [75, 76]}

In [173]:
remaining_indices_pass2

[22, 39, 55, 56, 67, 75, 76]

In [172]:
{p:[i for i in remaining_indices_pass2 if poly_eval_mod(data[i]['coefs'][::-1],0,p)==0] for p in ps_for_last}

{17: [22, 39, 75, 76],
 29: [39, 56],
 47: [22, 55, 67, 76],
 59: [67, 75],
 71: [39, 75]}

In [166]:
def find_easy_ss_roots(cs,p):
    cp = [c%p for c in cs]
    jsfound = []
    if p % 3 == 2 and cp[0]==0:
        jsfound.append(0)
    if p % 4 == 3:
        ev1 = poly_eval_mod(cp[::-1],1728%p,p)
        if ev1 == 0:
            jsfound.append(1728)
    return jsfound
    


In [176]:
last_assignments_i = {-71*4:75,-71:76,-236:67,-47:56,-47*4:55,-29*4:39,-17*4:22}
last_assignments={d:data[last_assignments_i[d]]['coefs'] for d in last_assignments}

In [177]:
last_assignments

{-284: [-709822597815616788561460999971455889888479357689323263,
  904330206367752622526324160408849797910089300094965,
  -282310474518147042483634599702449209357160103606,
  -12543980353722772760179537174109237891242365,
  -209903939873446020495519316596118272758,
  -4546951697981923176956448676986,
  -98373700603090523199715,
  1],
 -71: [737707086760731113357714241006081263,
  -425319473946139603274605151187659,
  5138800366453976780323726329446,
  -823534263439730779968091389,
  98394038810047812049302,
  -3091990138604570,
  313645809715,
  1],
 -236: [-528051042407719084320821843257565323032509500612428437651456,
  618701375850809570249968077855481255020992055993767559168,
  -674608806297136836505485361662125850918390913495465984,
  31142196350994346572482398388463886712080993091584,
  -2609998050684441805972918666809036652763152384,
  890259336020656022114227027503791079424,
  -85653894916610237425482419166691328,
  -8829246291887799055677742080,
  -911900087980611037808,
  1],


In [178]:
hilbpolys = {}
for d in disc_2_hcoefs:
    hilbpolys[d] = disc_2_hcoefs[d]
for d in last_assignments:
    hilbpolys[d] = last_assignments[d]

In [182]:
with open('../data/hilbpolys.json', 'w') as f:
    json.dump(hilbpolys, f, indent=4)

In [183]:
ssps

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 41, 47, 59, 71]